## Setup

In [ ]:
# Add scripts directory to path
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'scripts'))

# Import geometry functions
from geometry import (
    download_country_shapes,
    download_nuts3_shapes,
    join_shapes,
    point_in_shape,
    mask_shape,
    buffer_shape,
    get_shape_area,
    get_european_union_shape,
    load_shapes_efficiently,
    fill_from_boundary,
    to_point
)

# For visualization
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon

print("✓ Setup complete")

## 1. Download Country Shapes

Download boundaries for European countries. Data is cached locally after first download.

In [ ]:
# Download Germany and Poland
countries = download_country_shapes(['DE', 'PL'])

print(f"Downloaded {len(countries)} countries")
print("\nCountry data:")
countries[['country', 'geometry']]

In [ ]:
# Visualize
fig, ax = plt.subplots(figsize=(10, 8))
countries.plot(ax=ax, edgecolor='black', color=['blue', 'red'], alpha=0.5)
countries.apply(lambda x: ax.annotate(text=x['country'], 
                                       xy=x.geometry.centroid.coords[0],
                                       ha='center', fontsize=14, fontweight='bold'), axis=1)
plt.title('Germany and Poland', fontsize=16)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True, alpha=0.3)
plt.show()

## 2. Join Shapes

Combine multiple geometries into a single unified boundary.

In [ ]:
# Join Germany and Poland
combined = join_shapes(countries)

print(f"Combined shape type: {type(combined).__name__}")
print(f"Combined area: {get_shape_area(combined):,.0f} km²")

# Compare with individual areas
de_area = get_shape_area(countries[countries['country'] == 'DE'])
pl_area = get_shape_area(countries[countries['country'] == 'PL'])
print(f"\nGermany area:  {de_area:,.0f} km²")
print(f"Poland area:   {pl_area:,.0f} km²")
print(f"Sum of areas:  {de_area + pl_area:,.0f} km²")

In [ ]:
# Visualize combined shape
import geopandas as gpd

fig, ax = plt.subplots(figsize=(10, 8))
gpd.GeoDataFrame(geometry=[combined]).plot(ax=ax, color='green', alpha=0.5, edgecolor='black', linewidth=2)
plt.title('Combined Germany + Poland', fontsize=16)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True, alpha=0.3)
plt.show()

## 3. Point-in-Shape Checks

Test if coordinate points fall within boundaries.

In [ ]:
# Define cities with coordinates (lat, lon)
cities = {
    'Berlin': (52.5200, 13.4050),
    'Munich': (48.1351, 11.5820),
    'Warsaw': (52.2297, 21.0122),
    'Krakow': (50.0647, 19.9450),
    'Paris': (48.8566, 2.3522),
    'Prague': (50.0755, 14.4378),
}

# Check which cities are in our combined region
results = []
for city, (lat, lon) in cities.items():
    in_combined = point_in_shape(lat, lon, combined)
    results.append({'City': city, 'Latitude': lat, 'Longitude': lon, 'In DE+PL': in_combined})

results_df = pd.DataFrame(results)
print("\nCity location checks:")
print(results_df.to_string(index=False))

In [ ]:
# Visualize cities on map
fig, ax = plt.subplots(figsize=(12, 10))

# Plot countries
gpd.GeoDataFrame(geometry=[combined]).plot(ax=ax, color='lightblue', alpha=0.5, edgecolor='black', linewidth=2)

# Plot cities
for city, (lat, lon) in cities.items():
    in_region = point_in_shape(lat, lon, combined)
    color = 'green' if in_region else 'red'
    marker = 'o' if in_region else 'x'
    ax.plot(lon, lat, marker=marker, color=color, markersize=10, markeredgecolor='black', markeredgewidth=1)
    ax.annotate(city, (lon, lat), xytext=(5, 5), textcoords='offset points', fontsize=9, fontweight='bold')

plt.title('Cities in/out of DE+PL (Green=In, Red=Out)', fontsize=16)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True, alpha=0.3)
plt.show()

## 4. NUTS-3 Regions

Download and work with NUTS-3 statistical regions.

In [ ]:
# Download NUTS-3 regions for Germany
nuts3_de = download_nuts3_shapes(['DE'])

print(f"Downloaded {len(nuts3_de)} NUTS-3 regions for Germany")
print("\nSample regions:")
nuts3_de[['NUTS_ID', 'NAME_LATN']].head(10)

In [ ]:
# Visualize NUTS-3 regions
fig, ax = plt.subplots(figsize=(12, 10))
nuts3_de.plot(ax=ax, edgecolor='black', linewidth=0.5, cmap='tab20', alpha=0.7)
plt.title(f'German NUTS-3 Regions (n={len(nuts3_de)})', fontsize=16)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Find which NUTS-3 region contains Berlin
berlin = (52.5200, 13.4050)

for idx, row in nuts3_de.iterrows():
    if point_in_shape(berlin[0], berlin[1], row['geometry']):
        print(f"Berlin is in NUTS-3 region:")
        print(f"  ID: {row['NUTS_ID']}")
        print(f"  Name: {row['NAME_LATN']}")
        print(f"  Country: {row['CNTR_CODE']}")
        break

## 5. Intersection (Masking)

Compute the overlap between two boundaries.

In [ ]:
# Create a circular region around Berlin
from shapely.geometry import Point

berlin_point = Point(13.4050, 52.5200)
berlin_circle = berlin_point.buffer(1.0)  # ~100km radius

# Find intersection with Germany
germany = countries[countries['country'] == 'DE']
intersection = mask_shape(germany, berlin_circle, return_gdf=True)

print(f"Intersection area: {get_shape_area(intersection):,.0f} km²")
print(f"Original Germany area: {get_shape_area(germany):,.0f} km²")

In [ ]:
# Visualize intersection
fig, ax = plt.subplots(figsize=(12, 10))

# Plot Germany
germany.plot(ax=ax, color='lightblue', alpha=0.3, edgecolor='blue', linewidth=2, label='Germany')

# Plot circle
gpd.GeoDataFrame(geometry=[berlin_circle]).plot(ax=ax, color='yellow', alpha=0.3, edgecolor='orange', linewidth=2, label='100km around Berlin')

# Plot intersection
intersection.plot(ax=ax, color='red', alpha=0.5, edgecolor='darkred', linewidth=2, label='Intersection')

# Plot Berlin
ax.plot(13.4050, 52.5200, 'ko', markersize=10, label='Berlin')

plt.title('Intersection: Germany ∩ (100km radius around Berlin)', fontsize=16)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.show()

## 6. Buffer Operations

Expand or contract boundaries by a distance.

In [ ]:
# Download Luxembourg (small country)
luxembourg = download_country_shapes(['LU'])

# Create buffers
buffer_50km = buffer_shape(luxembourg, distance_km=50, return_gdf=True)
buffer_100km = buffer_shape(luxembourg, distance_km=100, return_gdf=True)

# Calculate areas
orig_area = get_shape_area(luxembourg)
area_50 = get_shape_area(buffer_50km)
area_100 = get_shape_area(buffer_100km)

print(f"Luxembourg area:        {orig_area:,.0f} km²")
print(f"+ 50km buffer:          {area_50:,.0f} km²  (+{area_50-orig_area:,.0f} km²)")
print(f"+ 100km buffer:         {area_100:,.0f} km²  (+{area_100-orig_area:,.0f} km²)")

In [ ]:
# Visualize buffers
fig, ax = plt.subplots(figsize=(10, 10))

buffer_100km.plot(ax=ax, color='lightblue', alpha=0.3, edgecolor='blue', linewidth=2, label='100km buffer')
buffer_50km.plot(ax=ax, color='lightgreen', alpha=0.3, edgecolor='green', linewidth=2, label='50km buffer')
luxembourg.plot(ax=ax, color='red', alpha=0.7, edgecolor='darkred', linewidth=2, label='Luxembourg')

plt.title('Luxembourg with Buffers', fontsize=16)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.show()

## 7. European Union Boundary

Get the unified boundary of all EU member states.

In [ ]:
# Get EU boundary
eu = get_european_union_shape()

print(f"EU shape type: {type(eu).__name__}")
print(f"EU total area: {get_shape_area(eu):,.0f} km²")

In [ ]:
# Check capital cities
capitals = {
    'Brussels (Belgium)': (50.8503, 4.3517),
    'Berlin (Germany)': (52.5200, 13.4050),
    'Paris (France)': (48.8566, 2.3522),
    'London (UK - not EU)': (51.5074, -0.1278),
    'Oslo (Norway - not EU)': (59.9139, 10.7522),
}

print("\nCapital cities in EU:")
for city, (lat, lon) in capitals.items():
    is_in_eu = point_in_shape(lat, lon, eu)
    status = "✓ In EU" if is_in_eu else "✗ Not in EU"
    print(f"  {city:30s}: {status}")

## 8. Practical Application: Filter Power Network Buses

Use the geometry module to filter buses by country.

In [ ]:
# Load buses data
buses_path = Path.cwd().parent / 'data' / 'raw' / 'OSM Prebuilt Electricity Network' / 'buses.csv'

if buses_path.exists():
    buses = pd.read_csv(buses_path)
    print(f"Loaded {len(buses)} buses")
    print("\nSample data:")
    print(buses[['bus_id', 'voltage', 'x', 'y', 'country']].head())
else:
    print(f"Buses file not found at {buses_path}")
    buses = None

In [ ]:
if buses is not None:
    # Filter buses in Germany using geometry
    germany_shape = download_country_shapes(['DE'])
    
    # Check first 100 buses (for speed)
    sample_buses = buses.head(100).copy()
    
    sample_buses['in_germany_geom'] = sample_buses.apply(
        lambda row: point_in_shape(row['y'], row['x'], germany_shape),
        axis=1
    )
    
    # Compare with country column
    sample_buses['in_germany_col'] = (sample_buses['country'] == 'DE')
    
    # Check agreement
    agreement = (sample_buses['in_germany_geom'] == sample_buses['in_germany_col']).sum()
    print(f"\nGeometry vs column agreement: {agreement}/{len(sample_buses)} ({100*agreement/len(sample_buses):.1f}%)")
    
    print(f"\nBuses in Germany (by geometry): {sample_buses['in_germany_geom'].sum()}")
    print(f"Buses in Germany (by column):   {sample_buses['in_germany_col'].sum()}")

## Summary

This notebook demonstrated:

1. ✅ **Downloading shapes** - Countries and NUTS-3 regions from Eurostat GISCO
2. ✅ **Joining shapes** - Union multiple geometries into one
3. ✅ **Point checks** - Test if coordinates fall within boundaries
4. ✅ **NUTS-3 regions** - Compatible with all geometry operations
5. ✅ **Intersection** - Compute overlap between boundaries
6. ✅ **Buffers** - Expand/contract boundaries by distance
7. ✅ **EU boundary** - Unified shape of all EU member states
8. ✅ **Practical use** - Filter power network elements by location

All data is automatically cached for fast subsequent access!

## 9. Europe-wide Visualization

Visualize the entire European power network infrastructure with countries, Voronoi regions, and bus locations.

In [ ]:
import json

JOIN = False

# Define cache dir
cache_dir = Path.cwd().parent / 'data' / 'cache' / 'geometry'

# Candidate paths (in preferred order)

all_countries_candidates = [
    cache_dir / 'all_countries.geojson',
    cache_dir / 'all_countries.parquet',
]
if JOIN:
    voronoi_candidates = [
        cache_dir / 'voronoi_eu27_join.geojson',
        cache_dir / 'voronoi_eu27_join.parquet',
    ]
else:
    voronoi_candidates = [
        cache_dir / 'voronoi_eu27.geojson',
        cache_dir / 'voronoi_eu27.parquet',
    ]

def first_existing(paths):
    for p in paths:
        if p.exists():
            return p
    return None

all_countries_path = first_existing(all_countries_candidates)
voronoi_eu27_path = first_existing(voronoi_candidates)

if all_countries_path and voronoi_eu27_path:
    # Load geometries
    all_countries_geojson = load_shapes_efficiently(all_countries_path)
    try: 
        all_countries_gdf = gpd.GeoDataFrame.from_features(all_countries_geojson['features'])
    except Exception as e:
        print(f"⚠ Failed to load all_countries GeoDataFrame: {e}")
        all_countries_gdf = all_countries_geojson
    
    voronoi_geojson = load_shapes_efficiently(voronoi_eu27_path)
    try:
        voronoi_gdf = gpd.GeoDataFrame.from_features(voronoi_geojson['features'])
    except Exception as e:
        print(f"⚠ Failed to load voronoi_eu27 GeoDataFrame: {e}")
        voronoi_gdf = voronoi_geojson
    
    # Load buses
    buses_path = Path.cwd().parent / 'data' / 'raw' / 'OSM Prebuilt Electricity Network' / 'buses.csv'
    if buses_path.exists():
        buses_data = pd.read_csv(buses_path)
        print(f"✓ Loaded {len(all_countries_gdf)} countries")
        print(f"✓ Loaded {len(voronoi_gdf)} Voronoi cells")
        print(f"✓ Loaded {len(buses_data)} buses")
    else:
        buses_data = None
        print(f"⚠ Buses file not found at {buses_path}")
else:
    print("⚠ Cache files not found")
    print("  Tried all_countries candidates:")
    for p in all_countries_candidates:
        print(f"    - {p} (exists={p.exists()})")
    print("  Tried voronoi candidates:")
    for p in voronoi_candidates:
        print(f"    - {p} (exists={p.exists()})")
    buses_data = None


In [ ]:
v_startswith = np.vectorize(lambda bus_id,y: bus_id.startswith(y))

EU_COUNTRIES = [
    'AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 'DK', 'EE', 'FI', 'FR',
    'DE', 'GR', 'HU', 'IE', 'IT', 'LV', 'LT', 'LU', 'MT', 'NL',
    'PL', 'PT', 'RO', 'SK', 'SI', 'ES', 'SE'
]

#display(buses)

In [ ]:
max_shape_bus_id = voronoi_gdf['bus_id'][np.where(voronoi_gdf['area_km2'] == voronoi_gdf['area_km2'].max())[0][0]]

buses_not_in_voronoi = pd.DataFrame()
for bus_id in buses_data['bus_id']:
    if not any(voronoi_gdf['bus_id'].apply(lambda x: bus_id == x)):
        buses_not_in_voronoi = pd.concat([buses_not_in_voronoi, buses_data[buses_data['bus_id'] == bus_id]], ignore_index=True)

buses_not_in_voronoi_eu = buses_not_in_voronoi[buses_not_in_voronoi['country'].isin(EU_COUNTRIES)].sort_values('country')

display(buses_not_in_voronoi_eu)

In [ ]:
point_vectorized = np.vectorize(to_point)

In [ ]:
if buses_data is not None:
    # Adjust font size for better readability
    plt.rcParams.update({'font.size': 4})
    # Create figure with larger size for detailed Europe view
    fig, ax = plt.subplots(figsize=(20,40), dpi=300)
    
    # Plot all countries with subtle color
    all_countries_gdf.plot(ax=ax, color='lightgray', edgecolor='darkgray', linewidth=0.5, alpha=0.6, figsize=(10,20))
    
    # Plot Voronoi cells with colors
    voronoi_gdf.plot(ax=ax, alpha=0.3, edgecolor='blue', linewidth=0.3, cmap='tab20b', figsize=(10,20))
    
    buses_not_in_voronoi = buses_not_in_voronoi[buses_not_in_voronoi['country'].isin(EU_COUNTRIES)]
    # Plot buses as small dots
    ax.scatter(voronoi_gdf['bus_x'], voronoi_gdf['bus_y'], c='green', s=5, alpha=1, label='Buses voronoi', zorder=5)
    ax.scatter(buses_not_in_voronoi['x'], buses_not_in_voronoi['y'], c='red', s=5, alpha=0.7, label='Sea Buses', zorder=5)
    
    # Plot also bus with max area of voronoi shape
    special_buses = buses[v_startswith(buses['bus_id'], max_shape_bus_id)]
    ax.scatter(special_buses['x'], special_buses['y'], c='orange', s=100, alpha=1.0, label='Bus with max area', edgecolor='black', zorder=6)
    
    
    # Formatting
    ax.set_title('European Power Network: Countries, Voronoi Cells (EU27), and Buses', fontsize=18, fontweight='bold', pad=20)
    ax.set_xlabel('Longitude', fontsize=12)
    ax.set_ylabel('Latitude', fontsize=12)

    ax.legend(loc='upper left', fontsize=11)
    ax.grid(True, alpha=0.2, linestyle='--')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ Visualization complete:")
    print(f"  • Gray regions: All European countries")
    print(f"  • Colored polygons: Voronoi cells for EU27 buses")
    print(f"  • Red dots: Power network buses ({len(buses_data)} total)")
else:
    print("Cannot create visualization - missing data files")


In [ ]:
def find_repo_root(max_up=6):
    p = Path.cwd().resolve()
    for _ in range(max_up):
        if (p / 'README.md').exists() or (p / '.git').exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    return Path.cwd().resolve()

repo_root = find_repo_root()

print(f"Repository root: {repo_root}")

In [ ]:
'''from pypsa_simplified import calculate_population_voronoi
from multiprocessing import Pool

pool = Pool()

parquet_path = repo_root / "data" / "processed" / "jrc_population_nonzero.parquet"

out = calculate_population_voronoi(parquet_path, voronoi_eu27_path)
'''


In [ ]:
if buses_data is not None:
    # Statistics
    print("\n📊 Network Statistics:")
    print(f"  Total buses: {len(buses_data)}")
    print(f"  Voltage levels: {sorted(buses_data['voltage'].unique())}")
    print(f"  Countries represented: {buses_data['country'].nunique()}")
    print(f"\nBuses by country (top 10):")
    print(buses_data['country'].value_counts().head(10))
    
    # Voronoi statistics
    print(f"\n  Voronoi EU27 cells: {len(voronoi_gdf)}")
    
    # Bounding box info
    print(f"\n🌍 Geographic extent:")
    print(f"  Longitude range: [{buses_data['x'].min():.2f}, {buses_data['x'].max():.2f}]")
    print(f"  Latitude range: [{buses_data['y'].min():.2f}, {buses_data['y'].max():.2f}]")


## 10. Test Improved Voronoi Generation

The `get_voronoi` function has been improved to:
1. **Filter sea buses**: Automatically remove buses located in the sea
2. **Add mirror points**: Create bounded Voronoi cells for all buses, including those near borders
3. **Fallback cells**: Generate small buffer cells for any buses without proper Voronoi cells

Let's regenerate the Voronoi diagram with these improvements.

In [ ]:
# Import required modules
from pypsa_simplified import data_prep
from geometry import get_voronoi, EU27

# Load OSM data
osm_dir = repo_root / "data" / "raw" / "OSM Prebuilt Electricity Network"
if osm_dir.exists():
    print("Loading OSM network data...")
    data_dict = data_prep.prepare_osm_source(osm_dir)
    raw_data = data_prep.RawData(data_dict)
    
    print(f"Buses loaded: {len(raw_data.data['buses'])}")
    print(f"EU27 countries: {EU27}")
else:
    print(f"⚠ OSM directory not found: {osm_dir}")
    raw_data = None

In [ ]:
if raw_data is not None:
    # Generate improved Voronoi diagram
    print("\n" + "="*60)
    print("Generating improved Voronoi diagram with:")
    print("  - Sea bus filtering")
    print("  - Mirror points for bounded cells")
    print("  - Fallback cells for edge cases")
    print("="*60 + "\n")
    
    voronoi_improved, mapping_improved = get_voronoi(
        raw_data, 
        countries=EU27, 
        join=False,
        add_mirror_points=True
    )
    
    print(f"\n✓ Created {len(voronoi_improved)} Voronoi cells")
    print(f"  - Bounded cells: {(voronoi_improved['bounded']==True).sum()}")
    print(f"  - Fallback cells: {(voronoi_improved['bounded']==False).sum()}")
    
    # Save to cache
    from geometry import save_shapes_efficiently
    output_path = repo_root / "data" / "cache" / "geometry" / "voronoi_eu27_improved"
    saved_path = save_shapes_efficiently(voronoi_improved, output_path)
    print(f"\n✓ Saved to: {saved_path}")
    
    # Save mapping
    mapping_path = repo_root / "data" / "cache" / "geometry" / "voronoi_eu27_mapping_improved.csv"
    mapping_improved.to_csv(mapping_path, index=False)
    print(f"✓ Saved mapping to: {mapping_path}")
else:
    print("⚠ Cannot generate Voronoi - no data loaded")
    voronoi_improved = None

In [ ]:
if voronoi_improved is not None:
    # Compare with old Voronoi (if available)
    if voronoi_gdf is not None:
        print("\n📊 Comparison: Old vs Improved Voronoi")
        print("="*60)
        print(f"Old method cells:      {len(voronoi_gdf)}")
        print(f"Improved method cells: {len(voronoi_improved)}")
        print(f"Difference:            +{len(voronoi_improved) - len(voronoi_gdf)} cells")
        
        # Find buses that now have cells
        old_buses = set(voronoi_gdf['bus_id'])
        new_buses = set(voronoi_improved['bus_id'])
        added_buses = new_buses - old_buses
        
        if added_buses:
            print(f"\n✓ {len(added_buses)} buses now have Voronoi cells that didn't before!")
            print(f"  Example buses: {list(added_buses)[:5]}")
    
    # Statistics
    print("\n📊 Improved Voronoi Statistics:")
    print("="*60)
    print(f"Total cells: {len(voronoi_improved)}")
    print(f"Bounded cells: {(voronoi_improved['bounded']==True).sum()} ({100*(voronoi_improved['bounded']==True).sum()/len(voronoi_improved):.1f}%)")
    print(f"Fallback cells: {(voronoi_improved['bounded']==False).sum()} ({100*(voronoi_improved['bounded']==False).sum()/len(voronoi_improved):.1f}%)")
    print(f"\nArea statistics:")
    print(f"  Total area: {voronoi_improved['area_km2'].sum():,.0f} km²")
    print(f"  Mean cell area: {voronoi_improved['area_km2'].mean():,.0f} km²")
    print(f"  Median cell area: {voronoi_improved['area_km2'].median():,.0f} km²")
    print(f"  Max cell area: {voronoi_improved['area_km2'].max():,.0f} km²")
    print(f"  Min cell area: {voronoi_improved['area_km2'].min():,.2f} km²")

In [ ]:
# From voronoi: 
# display(voronoi_improved)
# We collect the rows of buses that are in buses but not in voronoi_improved
buses_not_voronoi = pd.DataFrame()
for idx, row in buses.iterrows():
    if not any(voronoi_improved['bus_id'].apply(lambda x: row['bus_id'] == x)):
        buses_not_voronoi = pd.concat([buses_not_voronoi, pd.DataFrame([row])], ignore_index=True)

In [ ]:
buses_not_voronoi_EU = buses_not_voronoi[buses_not_voronoi['country'].isin(EU_COUNTRIES)]
europes_extremes = Polygon([(-12,72),(40.3,72),(40.3,34),(-12,34)])

In [ ]:
if voronoi_improved is not None and buses_data is not None:
    # Visualize improved Voronoi diagram
    fig, ax1 = plt.subplots(figsize=(10, 20))
    
    # Left: Overview
    all_countries_gdf.plot(ax=ax1, color='lightgray', edgecolor='darkgray', linewidth=0.5, alpha=0.6)
    voronoi_improved.plot(ax=ax1, alpha=0.3, edgecolor='blue', linewidth=0.3, cmap='tab20b')
    
    ax1.scatter(voronoi_improved['bus_x'], voronoi_improved['bus_y'], c='green', s=5, alpha=1, label='Buses in Voronoi', zorder=5)
    
    ax1.scatter(buses_not_voronoi_EU['x'], buses_not_voronoi_EU['y'], c='red', s=5, alpha=0.7, label='Buses not in Voronoi', zorder=5)
    
    ax1.plot(europes_extremes.exterior.xy[0], europes_extremes.exterior.xy[1], color='black', linestyle='--', linewidth=1, label='Europe Extent', zorder=7)
    
    max_shape_bus_id_improved = voronoi_improved['bus_id'][np.where(voronoi_improved['area_km2'] == voronoi_improved['area_km2'].max())[0][0]]
    special_buses_improved = buses[v_startswith(buses['bus_id'], max_shape_bus_id_improved)]
    #ax1.scatter(special_buses_improved['x'], special_buses_improved['y'], c='orange', s=100, alpha=1.0, label='Special Buses', edgecolor='black', zorder=6)
    
    # Mark bounded vs fallback cells
    '''
    bounded = voronoi_improved[voronoi_improved['bounded'] == True]
    fallback = voronoi_improved[voronoi_improved['bounded'] == False]
    
    ax1.scatter()
    ax1.scatter(bounded['bus_x'], bounded['bus_y'], c='green', s=5, alpha=0.8, label=f'Bounded cells ({len(bounded)})', zorder=5)
    ax1.scatter(fallback['bus_x'], fallback['bus_y'], c='orange', s=10, alpha=1.0, label=f'Fallback cells ({len(fallback)})', zorder=6)
    '''
    ax1.set_title('Improved Voronoi Diagram - Full EU27', fontsize=16, fontweight='bold')
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    #ax1.set_xlim(-12, 40.3)
    #ax1.set_ylim(34, 72)
    #ax1.set_xlim(-2.5,0)
    #ax1.set_ylim(42, 45)
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.2)
    
    
    plt.tight_layout()
    plt.show()
    
    print("\n✓ Visualization complete!")
    print("  Left: Full EU27 with all Voronoi cells")
    print("  Right: Zoomed to Northern Finland (previously had gaps)")

## Summary of Improvements

### What was fixed:

1. **Sea Bus Filtering** ✓
   - Buses located in the sea (outside country boundaries) are now automatically removed
   - Uses `point_in_shape()` check with actual country geometries
   - Prevents meaningless Voronoi cells in ocean areas

2. **Bounded Voronoi Cells** ✓
   - Added "mirror points" outside the boundary to ensure all interior points get bounded cells
   - Mirror points create a grid around the study area
   - Scipy's Voronoi no longer produces infinite regions for border buses

3. **Fallback Cell Generation** ✓
   - Buses that still don't get proper Voronoi cells (rare edge cases) get small buffer polygons
   - Ensures every bus has a cell for demand allocation
   - Marked with `bounded=False` flag for transparency

4. **Improved Clipping** ✓
   - All Voronoi cells properly clipped to country boundaries
   - No cells extending into the sea or outside EU27
   - Invalid geometries automatically fixed with `.buffer(0)`

### Result:
- **No more grey gaps** in Finland or other border regions
- **Every bus** now has a cell for demand allocation
- **Better coverage** near coastlines and national borders
- **Transparent**: `bounded` flag indicates fallback cells